### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Osteoporosis/GSE224330'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

import numpy as np

# Define sample_characteristics
sample_characteristics = {0: ['tissue: Isolated monocytes'], 1: ['age: 63y', 'age: 64y', 'age: 48y', 'age: 70y', 'age: 62y', 'age: 58y', 'age: 57y', 'age: 60y', 'age: 52y', 'age: 51y', 'age: 53y', 'age: 56y', 'age: 54y', 'age: 61y', 'age: 55y', 'age: 65y', 'age: 84y', 'age: 76y', 'age: 73y', 'age: 71y', 'age: 59y', 'age: 47y'], 2: ['gender: female', 'gender: male'], 3: ['comorbidity: hypothyroidism', 'comorbidity: none', 'comorbidity: osteoporosis', np.nan, 'comorbidity: schizoaffective disorder\xa0', 'comorbidity: arthrosis']}

# Check if gene expression data is available
is_gene_available = True

# Determine availability of variables
for key, values in sample_characteristics.items():
    unique_values = set(values)
    if any('osteoporosis' in v.lower() for v in unique_values):
        trait_row = key if len(unique_values) > 1 else None
    if any('age' in v.lower() for v in unique_values):
        age_row = key if len(unique_values) > 1 else None
    if any('gender' in v.lower() for v in unique_values):
        gender_row = key if len(unique_values) > 1 else None

# Define conversion functions
def convert_trait(value):
    if value:
        actual_value = value.split(':')[-1].strip().lower()
        if 'osteoporosis' in actual_value:
            return 1
        elif 'none' in actual_value:
            return 0
        else:
            return None
    return None

def convert_age(value):
    if value:
        try:
            return float(value.split(':')[-1].strip().rstrip('y'))
        except ValueError:
            return None
    return None

def convert_gender(value):
    if value:
        actual_value = value.split(':')[-1].strip().lower()
        if actual_value == 'female':
            return 0
        elif actual_value == 'male':
            return 1
        else:
            return None
    return None

# Save cohort information
save_cohort_info('GSE224330', './preprocessed/Osteoporosis/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical feature extraction conditionally
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Osteoporosis', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Osteoporosis/trait_data/GSE224330.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
